In [1]:
from __future__ import print_function
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
import os

Using TensorFlow backend.


In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:2 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:3 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:3 -> device: 3, name: GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1



In [3]:
batch_size = 256
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20

In [4]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 strides=1,
                 activation='relu',
                 input_shape=x_train.shape[1:]
                ))
model.add(Conv2D(32, (3, 3), strides=1, activation='relu'))
model.add(Conv2D(64, (3, 3), strides=2, activation='relu'))
model.add(Conv2D(64, (3, 3), strides=1, activation='relu'))
model.add(Conv2D(64, (3, 3), strides=1, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

W0204 15:09:31.662342 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0204 15:09:31.665385 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0204 15:09:31.670738 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [7]:
# initiate Adam optimizer
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

W0204 15:09:32.796570 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0204 15:09:32.803029 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=int(np.ceil(x_train.shape[0]/batch_size)),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

W0204 15:09:39.201055 140099731035968 deprecation.py:323] From /home/jiameng/packages/Bernsp/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Using real-time data augmentation.


W0204 15:09:39.262440 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0204 15:09:39.427077 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0204 15:09:39.611423 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Epoch 1/100


W0204 15:09:40.117464 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0204 15:09:40.119988 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0204 15:09:40.125295 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

W0204 15:09:40.663297 140099731035968 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



196/196 [==============================] - 28s 144ms/step - loss: 2.1553 - acc: 0.1521 - val_loss: 1.9163 - val_acc: 0.2329
Epoch 2/100
196/196 [==============================] - 26s 134ms/step - loss: 1.9052 - acc: 0.2322 - val_loss: 1.8495 - val_acc: 0.2663
Epoch 3/100
196/196 [==============================] - 26s 134ms/step - loss: 1.8282 - acc: 0.2906 - val_loss: 1.7543 - val_acc: 0.3185
Epoch 4/100
196/196 [==============================] - 26s 135ms/step - loss: 1.7424 - acc: 0.3303 - val_loss: 1.6195 - val_acc: 0.3742
Epoch 5/100
196/196 [==============================] - 26s 135ms/step - loss: 1.6085 - acc: 0.3903 - val_loss: 1.5237 - val_acc: 0.4277
Epoch 6/100
196/196 [==============================] - 26s 132ms/step - loss: 1.4917 - acc: 0.4464 - val_loss: 1.3900 - val_acc: 0.4964
Epoch 7/100
196/196 [==============================] - 26s 134ms/step - loss: 1.3836 - acc: 0.4878 - val_loss: 1.2798 - val_acc: 0.5331
Epoch 8/100
196/196 [==============================] - 26s 1

Epoch 61/100
196/196 [==============================] - 26s 134ms/step - loss: 0.4844 - acc: 0.8340 - val_loss: 0.6234 - val_acc: 0.7967
Epoch 62/100
196/196 [==============================] - 26s 135ms/step - loss: 0.4693 - acc: 0.8397 - val_loss: 0.6511 - val_acc: 0.7898
Epoch 63/100
196/196 [==============================] - 26s 132ms/step - loss: 0.4758 - acc: 0.8380 - val_loss: 0.6851 - val_acc: 0.7832
Epoch 64/100
196/196 [==============================] - 26s 133ms/step - loss: 0.4716 - acc: 0.8402 - val_loss: 0.6636 - val_acc: 0.7842
Epoch 65/100
196/196 [==============================] - 26s 133ms/step - loss: 0.4674 - acc: 0.8410 - val_loss: 0.6358 - val_acc: 0.7944
Epoch 66/100
196/196 [==============================] - 26s 134ms/step - loss: 0.4670 - acc: 0.8419 - val_loss: 0.6652 - val_acc: 0.7887
Epoch 67/100
196/196 [==============================] - 26s 135ms/step - loss: 0.4666 - acc: 0.8411 - val_loss: 0.6152 - val_acc: 0.8014
Epoch 68/100
196/196 [===================

In [10]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CIFAR_CNN_Large.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CIFAR_CNN_Large.h5")
print("Saved model to disk")

Saved model to disk
